In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import json

In [3]:
!pip install scrapy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.4/286.4 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.1/247.1 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.7/97.7 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 kB 8.5 MB/s eta 0:00:00


In [ ]:
#!scrapy startproject tutorial

In [ ]:
#!scrapy crawl quotes

In [4]:
%cd /content/drive/MyDrive/CS114/tutorial

/content/drive/MyDrive/CS114/tutorial


In [ ]:
import scrapy

class NewsSpider(scrapy.Spider):
    name = "news"

    def start_requests(self):
        urls = [
            'https://vnexpress.net/phap-luat'
        ]
        for url in urls:
            yield scrapy.Request(url, callback=self.parse)

    def parse(self, response):
        allLinks = response.css('h3.title-news a::attr(href)').getall()

        for each_link in allLinks:
            yield scrapy.Request(each_link, callback=self.parse_href)

        next_page = response.css('a.btn-page next-page::attr(href)').get()
        if next_page is not None:
            next_page = response.urljoin(next_page)
            yield scrapy.Request(next_page, callback=self.parse)

    def parse_href(self, response):
        title = response.css('.title-detail::text').get()
        title = title.replace('\n', '')
        title = title.replace('\t', '')
        paragraph = response.css('.fck_detail p::text').getall()
        text = ''
        count = 0
        for sentence in paragraph:
            if count == 4:
                break
            sentence = sentence.replace('\xa0', '')
            text = text + sentence
            count += 1
        yield {
            'title': title,
            'text': text,
            'is_negative': 1
        }


In [5]:
!scrapy crawl news -o phapluat.json

2024-01-14 04:12:56 [scrapy.utils.log] INFO: Scrapy 2.11.0 started (bot: tutorial)
2024-01-14 04:12:56 [scrapy.utils.log] INFO: Versions: lxml 4.9.4.0, libxml2 2.10.3, cssselect 1.2.0, parsel 1.8.1, w3lib 2.1.2, Twisted 22.10.0, Python 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0], pyOpenSSL 23.3.0 (OpenSSL 3.1.4 24 Oct 2023), cryptography 41.0.7, Platform Linux-6.1.58+-x86_64-with-glibc2.35
2024-01-14 04:12:56 [scrapy.addons] INFO: Enabled addons:
[]
2024-01-14 04:12:56 [asyncio] DEBUG: Using selector: EpollSelector
2024-01-14 04:12:56 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.asyncioreactor.AsyncioSelectorReactor
2024-01-14 04:12:56 [scrapy.utils.log] DEBUG: Using asyncio event loop: asyncio.unix_events._UnixSelectorEventLoop
2024-01-14 04:12:56 [scrapy.extensions.telnet] INFO: Telnet Password: 28a74eb789facc0d
2024-01-14 04:12:56 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 's

In [6]:
data = pd.read_json('/content/drive/MyDrive/CS114/tutorial/dataset-phapluat.json')
data

,title,text,is_negative
0,Vợ của 'nghi can cướp tiệm vàng tử vong trên đ...,. Bị can là nhân viên quản lý kho hàng của - ...,1
1,Chủ tịch Louis Holdings Đỗ Thành Nhân được giả...,Tòa cũng xác định số tiền thu bất chính từ vụ ...,1
2,Bảy thiếu niên tham gia vụ đánh chết nam sinh ...,"Chiều 7/1, 7 người hẹn Nguyễn Anh Tuấn (18 tuổ...",1
3,'Hot girl' miền Tây thuê nhà nghỉ bán ma túy,Kiều Thi Loan Em khi bị bắt quả tang tàng trữ ...,1
4,3 người chết cháy ở Gia Lai vì bị phóng hỏa,Hiện trường vụ cháy rạng sáng 9/1. Ảnh: Khi đá...,1
...,...,...,...
574,"Những lần hối lộ, bán độ của HLV nổi tiếng Tru...","Tháng 1/2020, Lý trở thành HLV trưởng của đội ...",1
575,Cựu bộ trưởng Y tế Nguyễn Thanh Long ra tòa,Cảnh sát dẫn giải các bị cáo từ hai xe 16 và 2...,1
576,Chủ tịch tỉnh Lâm Đồng Trần Văn Hiệp bị bắt,theo khoản 4 Điều 354 Bộ luật Hình sự. Vụ án t...,1
577,Dụ hàng chục học sinh vay 'nóng' mua điện thoại,Trần Huy Quốc tại cơ quan điều tra. Ảnh:Theo đ...,1
